In [1]:
import pandas as pd
import numpy as np
from benchmarks import *
import glob
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from tqdm.notebook import tqdm

ROOT_DIR /usr/WS2/bolet1/ruby-benchmarks/exploreGlobalOptimizations


In [2]:
MACHINE = 'lassen' if 'lassen' in ROOT_DIR else 'ruby'
print(MACHINE, ROOT_DIR)
prognames = list(progs.keys())
probsizes = ['smlprob', 'medprob', 'lrgprob']

goMethods=['cma', 'pso', 'bo']
seeds = [1337, 3827, 9999, 4873]

hypers = {
	'cma':['popsize', ('sigma', 'seed')],
	'pso':[('popsize', 'w'), ('c1', 'c2', 'seed')],
	'bo-ucb':['kappa', 'seed']
	'bo-ei': ['xi', 'seed']
	'bo-poi':['xi', 'seed']
}

ruby /usr/WS2/bolet1/ruby-benchmarks/exploreGlobalOptimizations


In [3]:
dbFile = f'{MACHINE}-fullExploreDataset.csv'
xtimeDB = pd.read_csv(ROOT_DIR+'/databases/'+dbFile)

globalOptimals = xtimeDB.groupby(['progname', 'probsize'])['xtime'].min().reset_index()


numthreads = 112 if MACHINE in 'ruby' else 160
globalBaselines = xtimeDB.loc[(xtimeDB['OMP_NUM_THREADS'] == numthreads) 
										& (xtimeDB['OMP_PROC_BIND'] == 'close')
										& (xtimeDB['OMP_PLACES'] == 'threads')
										& (xtimeDB['OMP_SCHEDULE'] == 'static'),['progname', 'probsize', 'xtime']]

probsizeMap = {'smlprob': 'Small Problem', 'medprob': 'Medium Problem', 'lrgprob': 'Large Problem'}
prognameMap = {'bt_nas': 'BT', 'ft_nas': 'FT', 'hpcg': 'HPCG', 'lulesh':'Lulesh'}

In [4]:
overallDF = pd.DataFrame()
tojoin = []
for progname in prognames:
	if progname == 'cg_nas' or progname == 'cfd_rodinia':
		continue
	for method in goMethods:
		# read the pre-processed dataframes
		filename = ROOT_DIR+'/databases/'+f'{MACHINE}-{progname}-{method}-GO_Data-rawXtimes.csv'
		fullDF = pd.read_csv(filename)
		tojoin += [fullDF]

overallDF = pd.concat(tojoin, ignore_index=True, sort=True)
overallDF = overallDF.drop(['optimXtime', 'kappa_decay', 'kappa_decay_delay'], axis=1)

print(overallDF.columns)
overallDF.loc[overallDF['method'] == 'bo', 'method'] = overallDF[overallDF['method'] == 'bo'].apply(lambda x: x['method']+'-'+x['utilFnct'], axis=1)
overallDF = overallDF.drop(['utilFnct'], axis=1)

for col in overallDF:
	if col == 'xtime' or col == 'globalSample' or col == 'optimXtime':
		continue
	print(col, overallDF[col].unique())

Index(['c1', 'c2', 'globalSample', 'kappa', 'method', 'popsize', 'probsize',
       'progname', 'seed', 'sigma', 'utilFnct', 'w', 'xi', 'xtime'],
      dtype='object')
c1 [ nan 0.1  0.45 0.8  1.15 1.5 ]
c2 [ nan 0.1  0.45 0.8  1.15 1.5 ]
kappa [ nan   2.   8.  15.  22.  29.  36.  42.  49.  56.  63.  70.  77.  83.
  90.  97. 104. 111. 118. 124. 131. 138. 145. 152. 159. 165. 172. 179.
 186. 193. 200.]
method ['cma' 'pso' 'bo-ei' 'bo-ucb' 'bo-poi']
popsize [ 3.  6.  9. 12. 15. 18. 21. 24. 27. 30. nan]
probsize ['smlprob' 'medprob' 'lrgprob']
progname ['bt_nas' 'ft_nas' 'hpcg' 'lulesh']
seed [1337 3827 9999 4873]
sigma [ 1.     4.222  7.444 10.667 13.889 17.111 20.333 23.556 26.778 30.
    nan]
w [  nan 0.1   0.325 0.55  0.775 1.   ]
xi [  nan 0.    0.357 0.714 1.071 1.429 1.786 2.143 2.5   2.857 3.214 3.571
 3.929 4.286 4.643 5.   ]


In [ ]:
def makeGOHyperHeatmapPlots(df, method):

	df = df[df['method'] == method].reset_index()

	# we're going to make one plot for each GO method
	# want to show programs in rows, probsize in columns

	# the heatmap for each will show the hyperparameters on the two axes
	# the heatmap values will be the first globalSample that on average beats out the baseline

	# let's first make the tuples of columns

	g = sns.FacetGrid(df, row='progname', col='probsize', col_order=probsizes, palette='flare', height=10, aspect=1.5)
	#g.map_dataframe(drawHeatmap, annot=True, vmin=0.0, vmax=1.0, cbar=True)
	g.map_dataframe(drawHeatmap, annot=True, cbar=True)


	for ax in g.axes.flatten():
		ax.tick_params(axis='x', labelbottom=True, labelrotation=90)

	plt.tight_layout()

	g.fig.subplots_adjust(top=0.96)
	g.fig.suptitle(f'OMP Hyperparam Exploration -- Speedups Over Baselines')

	plt.show()

	# for PSO we will need to make tuples
	return


In [ ]:
makeGOHyperHeatmapPlots(overallDF, 'cma')
makeGOHyperHeatmapPlots(overallDF, 'pso')
makeGOHyperHeatmapPlots(overallDF, 'bo-ucb')
makeGOHyperHeatmapPlots(overallDF, 'bo-ei')
makeGOHyperHeatmapPlots(overallDF, 'bo-poi')

In [ ]:

def makeOverallPlots(df, baselineXtimes=None):
	# want each row to be a program, and each column to be a problem size
	# each plot will show the average cumsum across each bo-util fnct, PSO, and CMA

	otherCols = list(df.columns)
	otherCols.remove('xtime')
	otherCols.remove('globalSample')
	print(otherCols)

	# for each category, we need to do a cumulative max
	df['cummax'] = df.groupby(otherCols, dropna=False)['xtime'].transform('cummax')

	def addBaselines(*args, **kwargs):
		data = kwargs['data']
		probsize = data.iloc[0]['probsize']
		progname = data.iloc[0]['progname']
		method = data.iloc[0]['method']

		ax = plt.gca()
		handles, labels = ax.get_legend_handles_labels()

		optimal = globalOptimals[(globalOptimals['progname'] == progname) & (globalOptimals['probsize'] == probsize)]['xtime'].iat[0]
		baseline = globalBaselines[(globalBaselines['progname'] == progname) & (globalBaselines['probsize'] == probsize)]['xtime'].iat[0]

		#avrgCummax = data.groupby(['globalSample'], dropna=False)['cummax'].mean().reset_index()
		#if baselineXtimes is None:
		#	#step = data[data['cummax'] >= 0.5]['globalSample'].min()
		#	step = avrgCummax[avrgCummax['cummax'] >= 0.5]['globalSample'].min()
		#	if step is None:
		#		print(f'{progname} {probsize} {method} did not reach baseline')
		#	else:
		#		print(f'{progname} {probsize} {method} passed baseline on step {step}')
		#else:
		#	val = baselineXtimes.loc[(baselineXtimes['progname'] == progname) & (baselineXtimes['probsize'] == probsize), 'xtime'].iat[0]
		#	poss = baselineXtimes.loc[(baselineXtimes['progname'] == progname) & (baselineXtimes['probsize'] == probsize), 'xtime']
		#	print('poss')
		#	print(poss)
		#	step = avrgCummax[avrgCummax['cummax'] >= val]['globalSample'].min()
		#	if step is None:
		#		print(f'{progname} {probsize} {method} did not reach baseline')
		#	else:
		#		print(f'{progname} {probsize} {method} passed baseline on step {step}')
		#	print('at step 280, avrgCummax is', avrgCummax[avrgCummax['globalSample'] == 280.0])

		if not ('optimal' in labels):
			ax.set_title(f'{prognameMap[progname]} -- {probsizeMap[probsize]}')
			ax.set_xlabel('Sample Index')

			if baselineXtimes is None:
				#ax.set_ylabel('Norm. to Optimal/Baseline Execution Times from Database \n(higher is better)')
				ax.axhline(baseline/optimal, c='blue', linestyle='--', zorder=0, label='optimal')
				ax.axhline(1.0, c='red', linestyle='--', zorder=0, label='baseline')
				# what step do we pass the baseline xtime?
			else:
				#ax.set_ylabel('Norm. to Optimal/Worst Execution Times from Database \n(higher is better)')
				val = baselineXtimes.loc[(baselineXtimes['progname'] == progname) & (baselineXtimes['probsize'] == probsize), 'xtime'].iat[0]
				ax.axhline(1.0, c='blue', linestyle='--', zorder=0, label='optimal')
				ax.axhline(val, c='red', linestyle='--', zorder=0, label='baseline')

		#ax.legend(loc='lower right')
		return

	g = sns.FacetGrid(df, row='progname', col='probsize', col_order=probsizes, hue='method', 
										palette='flare', legend_out=False, height=5, aspect=1.5, sharex=False, sharey=False)
	g.map_dataframe(sns.lineplot, x='globalSample', y='cummax', errorbar="pi")
	g.map_dataframe(addBaselines)
	#g.set(ylim=(-0.05, 1.05))

	axes = g.axes
	for r in range(axes.shape[0]):
		if baselineXtimes is None:
			axes[r,0].set_ylabel('Norm. to Baseline from Database (Speedup) \n( >1.0 is better )')
		else:
			axes[r,0].set_ylabel('Norm. to Optimal/Worst Execution Times from Database \n(higher is better)')
		for c in range(axes.shape[1]):
			axes[r,c].legend(loc='lower right')

	plt.tight_layout()

	g.fig.subplots_adjust(top=0.95)
	g.fig.suptitle('Average highest normalized execution time found at each optimization step')

	plt.show()
	return


In [ ]:

useBaseline=True

overallDF = pd.DataFrame()
tojoin = []
for progname in prognames:
	if progname == 'cg_nas' or progname == 'cfd_rodinia':
		continue
	for method in goMethods:
		# read the pre-processed dataframe
		if useBaseline:
			filename = ROOT_DIR+'/databases/'+f'{MACHINE}-{progname}-{method}-GO_Data-baselineNorm.csv'
		else:
			filename = ROOT_DIR+'/databases/'+f'{MACHINE}-{progname}-{method}-GO_Data.csv'
		fullDF = pd.read_csv(filename)
		tojoin += [fullDF]

overallDF = pd.concat(tojoin, ignore_index=True, sort=True)
overallDF = overallDF.drop(['optimXtime', 'kappa_decay', 'kappa_decay_delay'], axis=1)

print(overallDF.columns)
# for the method column, make all the bo entries a union of the method and utilFnct
#overallDF[overallDF['method'] == 'bo'].apply(lambda x: x['method']+'-'+x['utilFnct'], axis=1)
#print('it works')
overallDF.loc[overallDF['method'] == 'bo', 'method'] = overallDF[overallDF['method'] == 'bo'].apply(lambda x: x['method']+'-'+x['utilFnct'], axis=1)
#print(overallDF['method'].unique())

overallDF = overallDF.drop(['utilFnct'], axis=1)

for col in overallDF:
	if col == 'xtime' or col == 'globalSample' or col == 'optimXtime':
		continue
	print(col, overallDF[col].unique())

In [ ]:
baselinesDF = xtimeDB[(xtimeDB['OMP_NUM_THREADS'] == numthreads) 
										& (xtimeDB['OMP_PROC_BIND'] == 'close')
										& (xtimeDB['OMP_PLACES'] == 'threads')
										& (xtimeDB['OMP_SCHEDULE'] == 'static')]

def normToMinMax(row):
	progname = row['progname']
	probsize = row['probsize']

	minVal, maxVal = getMinMaxXtimeForProg(progname, probsize)

	row['xtime'] = 1 - (row['xtime']-minVal)/(maxVal-minVal)
	return row


# normalize the baselinesDF to the min/max
baselinesDF.loc[:,'xtime'] = baselinesDF.apply(normToMinMax, axis=1)['xtime']

if useBaseline:
	makeOverallPlots(overallDF)
else:
	makeOverallPlots(overallDF,baselinesDF)